In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

def get_session(gpu_fraction=0.3):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session(0.5))

Using TensorFlow backend.


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

import os

In [3]:
batch_size = 16
num_classes = 3
epochs = 1000

# num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_neuron_classification_trained_model.h5'

In [4]:
# write a function to select x_train y_train x_test y_test
data_x = np.load('image_data_256x256.npy')
data_y = np.load('labels.npy')
data_x = np.swapaxes(np.swapaxes(data_x,1,2),2,3)
data_y = np.array([data_y]).T

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)
print x_train.shape, x_test.shape, y_train.shape, y_test.shape
# shuffle the data
# select 20% as the test data and 80% as the training data
# np.random.choice(len(data_x), len(data_x) * 0.2, replace=False)

/home/edward/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(221, 256, 256, 2) (56, 256, 256, 2) (221, 1) (56, 1)


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print x_train.shape, x_test.shape, y_train.shape, y_test.shape

(221, 256, 256, 2) (56, 256, 256, 2) (221, 3) (56, 3)


In [7]:
# define the model 
# CNN structure
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [8]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:
data_augmentation = False

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
      
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        callbacks=None)

Not using data augmentation.
Train on 221 samples, validate on 56 samples
Epoch 1/1000
221/221 [==============================] - 3s - loss: 1.0045 - acc: 0.5520 - val_loss: 0.9628 - val_acc: 0.6250
Epoch 2/1000
221/221 [==============================] - 1s - loss: 0.9676 - acc: 0.5837 - val_loss: 0.9517 - val_acc: 0.6250
Epoch 3/1000
221/221 [==============================] - 1s - loss: 0.9606 - acc: 0.5837 - val_loss: 0.9148 - val_acc: 0.6250
Epoch 4/1000
221/221 [==============================] - 1s - loss: 0.9906 - acc: 0.5837 - val_loss: 0.9235 - val_acc: 0.6250
Epoch 5/1000
221/221 [==============================] - 1s - loss: 0.9808 - acc: 0.5837 - val_loss: 0.9395 - val_acc: 0.6250
Epoch 6/1000
221/221 [==============================] - 1s - loss: 0.9774 - acc: 0.5837 - val_loss: 0.9353 - val_acc: 0.6250
Epoch 7/1000
221/221 [==============================] - 1s - loss: 0.9809 - acc: 0.5837 - val_loss: 0.9433 - val_acc: 0.6250
Epoch 8/1000
221/221 [=============================

221/221 [==============================] - 1s - loss: 0.0030 - acc: 1.0000 - val_loss: 2.4559 - val_acc: 0.5179
Epoch 66/1000
221/221 [==============================] - 1s - loss: 0.0013 - acc: 1.0000 - val_loss: 2.6105 - val_acc: 0.5714
Epoch 67/1000
221/221 [==============================] - 1s - loss: 0.0052 - acc: 0.9955 - val_loss: 2.4773 - val_acc: 0.5536
Epoch 68/1000
221/221 [==============================] - 1s - loss: 8.8962e-04 - acc: 1.0000 - val_loss: 2.5964 - val_acc: 0.5536
Epoch 69/1000
221/221 [==============================] - 1s - loss: 8.9954e-04 - acc: 1.0000 - val_loss: 2.6417 - val_acc: 0.5357
Epoch 70/1000
221/221 [==============================] - 1s - loss: 0.0010 - acc: 1.0000 - val_loss: 2.6518 - val_acc: 0.5536
Epoch 71/1000
221/221 [==============================] - 1s - loss: 2.4027e-04 - acc: 1.0000 - val_loss: 2.8374 - val_acc: 0.5714
Epoch 72/1000
221/221 [==============================] - 1s - loss: 2.3914e-04 - acc: 1.0000 - val_loss: 2.8182 - val_ac

KeyboardInterrupt: 

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])